# PySpark DataFrame subsetting and cleaning

- After data inspection, it is often necessary to clean the data which mainly involves subsetting, renaming the columns, removing duplicated rows etc., PySpark DataFrame API provides several operators to do this. In this exercise, your job is to subset `'name'`, `'sex'` and `'date of birth'` columns from `people_df` DataFrame, remove any duplicate rows from that dataset and count the number of rows before and after duplicates removal step.

- Remember, you already have `SparkSession` `spark` and `people_df` DataFrames available in your workspace.

## Instructions

- Select `'name'`, `'sex'` and `'date of birth'` columns from `people_df` and create `people_df_sub` DataFrame.
- Print the first 10 observations in the `people_df` DataFrame.
- Remove duplicate entries from `people_df_sub` DataFrame and create `people_df_sub_nodup` DataFrame.
- How many rows are there before and after duplicates are removed?

In [1]:
# Intialization
import os
import sys

os.environ["SPARK_HOME"] = "/home/talentum/spark"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.6" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

# NOTE: Whichever package you want mention here.
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0 pyspark-shell' 
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.3 pyspark-shell'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell''

In [2]:
#Entrypoint 2.x
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().getOrCreate()

# On yarn:
# spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().master("yarn").getOrCreate()
# specify .master("yarn")

sc = spark.sparkContext

In [3]:
file_path = "file:///home/talentum/test-jupyter/P2/M3/SM2/Dataset/people.csv"

# Create an DataFrame from file_path
people_df = spark.read.csv(file_path, header=True, inferSchema=True)

# Select name, sex and date of birth columns
people_df_sub = people_df.select('name', 'sex', 'date of birth')

# Print the first 10 observations from people_df_sub
people_df_sub.show(10)

# Remove duplicate entries from people_df_sub
people_df_sub_nodup = people_df_sub.dropDuplicates()

# Count the number of rows
print("There were {} rows before removing duplicates, and {} rows after removing duplicates".format(people_df_sub.count(), people_df_sub_nodup.count()))

+----------------+------+-------------+
|            name|   sex|date of birth|
+----------------+------+-------------+
|  Penelope Lewis|female|   1990-08-31|
|   David Anthony|  male|   1971-10-14|
|       Ida Shipp|female|   1962-05-24|
|    Joanna Moore|female|   2017-03-10|
|  Lisandra Ortiz|female|   2020-08-05|
|   David Simmons|  male|   1999-12-30|
|   Edward Hudson|  male|   1983-05-09|
|    Albert Jones|  male|   1990-09-13|
|Leonard Cavender|  male|   1958-08-08|
|  Everett Vadala|  male|   2005-05-24|
+----------------+------+-------------+
only showing top 10 rows

There were 100000 rows before removing duplicates, and 99998 rows after removing duplicates


In [8]:
df = people_df_sub.groupBy('name', 'sex', 'date of birth').count()

In [27]:
df.filter('count > 1').show()

+-------------+------+-------------+-----+
|         name|   sex|date of birth|count|
+-------------+------+-------------+-----+
|Kathryn Davis|female|  20175-02-28|    2|
| Robert Smith|  male|  20175-02-28|    2|
+-------------+------+-------------+-----+



In [29]:
people_df_sub.groupBy(people_df_sub.name, people_df_sub.sex, people_df_sub.date of birth) # Column object

SyntaxError: invalid syntax (<ipython-input-29-84b6893fb820>, line 1)

In [32]:
people_df_sub.groupBy(people_df_sub.name, people_df_sub.sex, people_df_sub['date of birth'])\
.count()\
.filter('count > 1')\
.show() # Column object with column name

+-------------+------+-------------+-----+
|         name|   sex|date of birth|count|
+-------------+------+-------------+-----+
|Kathryn Davis|female|  20175-02-28|    2|
| Robert Smith|  male|  20175-02-28|    2|
+-------------+------+-------------+-----+



In [37]:
people_df_sub.groupBy(people_df_sub.name, people_df_sub.sex, people_df_sub['date of birth'])\
.count()\
.filter(people_df_sub.count > 1)\
.show()  # count considered as method here

TypeError: '>' not supported between instances of 'method' and 'int'

Using SQL Functions

In [41]:
import  pyspark.sql.functions as F

people_df.groupBy(people_df.name, people_df.sex, people_df['date of birth'])\
.count()\
.filter(F.col('count') > 1)\
.show()
# Considered as a more OOPS way

print(type(F.col('count') ))
print(type(F.col('count') > 1))

+-------------+------+-------------+-----+
|         name|   sex|date of birth|count|
+-------------+------+-------------+-----+
|Kathryn Davis|female|  20175-02-28|    2|
| Robert Smith|  male|  20175-02-28|    2|
+-------------+------+-------------+-----+

<class 'pyspark.sql.column.Column'>
<class 'pyspark.sql.column.Column'>


`df1.exceptAll(df2).show()`


In [44]:
people_df_sub.exceptAll(people_df_sub.dropDuplicates()).show()

+-------------+------+-------------+
|         name|   sex|date of birth|
+-------------+------+-------------+
|Kathryn Davis|female|  20175-02-28|
| Robert Smith|  male|  20175-02-28|
+-------------+------+-------------+

